In [1]:
import pandas as pd
import numpy as np
import re
import time
from datetime import datetime
import random
import math
from collections import defaultdict

import json
import tqdm
import os

import matplotlib.pyplot as plt
from matplotlib import cm as cmap
import seaborn as sns

import folium
import folium.plugins

import branca
import branca.colormap as cm

import warnings
warnings.filterwarnings("ignore")

%config Completer.use_jedi = False
sns.set()
plt.style.use('seaborn-whitegrid')
plt.rcParams['axes.grid'] = True

In [3]:
path = './data/03_cleaned_df/'

# Nome dos arquivos dentro do diretorio acima (raw_df)
filenames = os.listdir(path)

# Dataframe dos dados sem tratamento
df = pd.read_csv(path + filenames[-1], sep='|') # -1 para pegar o ultimo dataframe adquirido

# Criando coluna com a padrão do bairro (alta, média ou baixa)

In [4]:
bairros = df.groupby('bairro').mean()['price'].sort_values(ascending=False)
bairros

bairro
morro dos ingleses               4822.923077
santa cecilia                    3940.500000
varzea da barra funda            3600.000000
consolacao                       3283.808118
parque industrial tomas edson    2912.142857
aclimacao                        2570.000000
barra funda                      2524.922330
bela vista                       2485.918864
santa efigenia                   2449.875000
belenzinho                       2225.620690
republica                        2103.272727
cambuci                          2080.034247
se                               2040.064516
centro                           2000.892857
bom retiro                       1907.238095
vila buarque                     1899.923077
vila deodoro                     1837.500000
liberdade                        1688.247423
campos eliseos                   1518.682927
luz                              1500.307692
Name: price, dtype: float64

In [5]:
bairros = df.groupby('bairro').mean()['price'].sort_values(ascending=False)

split = 3 # Numeros de grupos 
n = len(bairros) // split # Tamanho de cada grupo

bairros[:n] = 3 # Classe alta
bairros[n:n*2] = 2 # Classe média
bairros[n*2:] = 1 # Classe baixa

df['padrao_bairro'] = df.replace({"bairro": bairros.to_dict()})['bairro']
df[['bairro', 'padrao_bairro']]

,bairro,padrao_bairro
0,campos eliseos,1.0
1,aclimacao,3.0
2,aclimacao,3.0
3,cambuci,2.0
4,cambuci,2.0
...,...,...
2166,vila buarque,1.0
2167,republica,2.0
2168,bela vista,2.0
2169,campos eliseos,1.0


# Preço Total (feature `price` + `condominio`)

In [7]:
df['price_total'] = df['price'] + df['condominio']

# Encoding o bairro

In [8]:
# df = pd.concat([df.drop('bairro', axis=1), pd.get_dummies(df['bairro'], drop_first=True)], axis=1) # get_dummies da feature bairro
df = pd.concat([df, pd.get_dummies(df['bairro'], drop_first=True)], axis=1) # get_dummies da feature bairro modelo riobson

# Tratamento das variaveis categoricas para o LSI & LDA

In [9]:
df_olx = df.copy() # Duplicando o dataframe para salvar separadamente

In [10]:
desc_1 = []
desc_1b = []
df_olx['apartment_features'].fillna('', inplace = True)
df_olx['apartment_complex_features'].fillna('', inplace = True)
df_olx['garage_spaces'].fillna(0, inplace = True)

for index, row in df_olx.iterrows():
    if (row['apartment_complex_features'] != ''):
        desc_1.append (str(row['apartment_features']).replace(' ','_').replace(',_',', ')+ ', '+ str(row['apartment_complex_features']).replace(' ','_').replace(',_',',')\
        + ', ' + str(row['rooms'])+'_quarto'\
        + ', ' + str(row['size'])+'_area'\
        + ', ' + str(row['bathrooms'])+'_banheiro'\
        + ', ' + str(row['garage_spaces'])+'_vaga'\
        + ', ' + str(row['bairro']))
                      
    else:
        if (row['apartment_features'] != ''):
            desc_1.append (str(row['apartment_features']).replace(' ','_').replace(',_',', ')\
            + ', ' + str(row['rooms'])+'_quarto'\
            + ', ' + str(row['size'])+'_area'\
            + ', ' + str(row['bathrooms'])+'_banheiro'\
            + ', ' + str(row['garage_spaces'])+'_vaga'\
            + ', ' + str(row['bairro']))
            
        else:
            desc_1.append(str(row['rooms'])+'_quarto'\
            + ', ' + str(row['rooms'])+'_quarto'\
            + ', ' + str(row['size'])+'_area'\
            + ', ' + str(row['bathrooms'])+'_banheiro'\
             + ', ' + str(row['garage_spaces'])+'_vaga'\
        + ', ' + str(row['bairro']))
df_olx['DESC_3'] = desc_1
print(desc_1[1])

area_de_servico, permitido_animais, 2_quarto, 79.99999999999997_area, 2_banheiro, 1_vaga, aclimacao


## Dropando colunas e preparando os dataframes para os modelos lda e rf

In [11]:
# Dropando colunas e preparando o dataframe para o rf
cols_df1 = ['title', 'logradouro', 'latitude', \
        'longitude', 'zipcode', 'source_x', 'altitude', 'sellername', 'state', 'region', 'complemento', \
        'bairro', 'apartment_complex_features_rf',
       'apartment_features_rf', 'apartment_complex_features', 'apartment_features', 'desc', 'iptu']

df_rf = df.drop(cols_df1, axis=1) # Dataframe modificado para o rf
df_rf = df_rf[~df_rf['list_id'].duplicated(keep='first')] # Dropando as colunas duplicadas

In [12]:
cols = ['list_id', 'link', 'title', 'price', 'condominio', 'rooms', 'bathrooms',
       'garage_spaces', 'size', 'sellername', 'desc', 'apartment_features',
       'pictures', 'apartment_complex_features', 'zipcode', 'state', 'region', 'bairro', 
        'logradouro', 'DESC_3', 'iptu']

df_lda = df_olx[cols]
df_lda = df_lda[~df_lda['list_id'].duplicated(keep='first')] # Dropando as colunas duplicadas

In [13]:
# Salvando os dfs
now = datetime.now().strftime("%Y-%m-%d_%H%M%S") # Data e horario
df_rf.to_csv(f'./data/04_feat_eng_df/feat_end_df_{now}_rf.csv', sep='|',  index=False) # Cria o dataframe rf
df_lda.to_csv(f'./data/04_feat_eng_df/feat_end_df_{now}_lda.csv', sep='|',  index=False) # Cria o dataframe lda